In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [41]:
polls2020 = pd.read_csv('https://projects.fivethirtyeight.com/polls-page/president_polls.csv', 
                        parse_dates=['start_date', 'end_date', 'created_at'])
polls2020.to_csv('..\\..\\data\\raw\\president_polls2020.csv')
polls2020

,question_id,poll_id,cycle,state,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,...,ranked_choice_reallocated,created_at,notes,url,stage,race_id,answer,candidate_name,candidate_party,pct
0,124511,66740,2020,Iowa,383,Public Policy Polling,32,Emily's List,Public Policy Polling,263.0,...,False,2020-06-05 09:16:00,NaN,https://secure.emilyslist.org/page/-/IowaResul...,general,6223,Biden,Joseph R. Biden Jr.,DEM,47.0
1,124511,66740,2020,Iowa,383,Public Policy Polling,32,Emily's List,Public Policy Polling,263.0,...,False,2020-06-05 09:16:00,NaN,https://secure.emilyslist.org/page/-/IowaResul...,general,6223,Trump,Donald Trump,REP,48.0
2,124517,66742,2020,Texas,383,Public Policy Polling,1328,Texas Democratic Party,Public Policy Polling,263.0,...,False,2020-06-05 11:28:00,NaN,https://www.texasdemocrats.org/wp-content/uplo...,general,6254,Biden,Joseph R. Biden Jr.,DEM,48.0
3,124517,66742,2020,Texas,383,Public Policy Polling,1328,Texas Democratic Party,Public Policy Polling,263.0,...,False,2020-06-05 11:28:00,NaN,https://www.texasdemocrats.org/wp-content/uplo...,general,6254,Trump,Donald Trump,REP,48.0
4,124521,66744,2020,NaN,1583,Whitman Insight Strategies,NaN,NaN,Whitman Insight Strategies,584.0,...,False,2020-06-05 12:34:00,NaN,https://twitter.com/mattmfm/status/12689434139...,general,6210,Biden,Joseph R. Biden Jr.,DEM,53.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4673,92081,57026,2020,Ohio,383,Public Policy Polling,918,DEM,Public Policy Polling,263.0,...,False,2018-12-06 14:45:00,NaN,https://www.cleveland.com/politics/2018/11/thi...,general,6246,Trump,Donald Trump,REP,46.0
4674,92082,57026,2020,Ohio,383,Public Policy Polling,918,DEM,Public Policy Polling,263.0,...,False,2018-12-06 14:46:00,NaN,https://www.cleveland.com/politics/2018/11/thi...,general,6246,Warren,Elizabeth Warren,DEM,43.0
4675,92082,57026,2020,Ohio,383,Public Policy Polling,918,DEM,Public Policy Polling,263.0,...,False,2018-12-06 14:46:00,NaN,https://www.cleveland.com/politics/2018/11/thi...,general,6246,Trump,Donald Trump,REP,49.0
4676,92078,57025,2020,NaN,399,Rasmussen Reports/Pulse Opinion Research,NaN,NaN,Rasmussen Reports/Pulse Opinion Research,277.0,...,False,2018-12-06 14:40:00,NaN,http://www.rasmussenreports.com/public_content...,general,6210,Obama,Michelle Obama,DEM,50.0


In [3]:
polls2020 = polls2020[polls2020['cycle'] == 2020]
polls2020 = polls2020[polls2020['office_type'] == 'U.S. President']
polls2020 = polls2020[polls2020['candidate_name'].isin(['Joseph R. Biden Jr.', 'Donald Trump'])]
polls2020['candidate_name'] = polls2020['candidate_name'].replace({'Joseph R. Biden Jr.':'Biden', 'Donald Trump':'Trump'})
polls2020['state'] = polls2020['state'].fillna('National')

cols = ['question_id', 'poll_id', 'state', 'pollster_id', 'pollster', 'sample_size', 
        'population', 'start_date', 'end_date', 'created_at', 'url', 'candidate_name', 'candidate_party', 'pct']
polls2020 = polls2020[cols]

In [7]:
polls_index = polls2020['question_id'].value_counts() == 2
polls_index = polls_index[polls_index.values].index

In [8]:
polls2020 = polls2020[polls2020['question_id'].isin(polls_index)]

In [9]:
polls2020_descriptive = polls2020.drop(['candidate_name', 'candidate_party', 'pct'], axis='columns').set_index(['poll_id', 'question_id'])

polls2020_values = polls2020[['poll_id', 'question_id', 'candidate_name', 'pct']]
polls2020_values = pd.pivot_table(polls2020_values, values=['pct'], columns=['candidate_name'], index=['poll_id', 'question_id'])

In [10]:
polls2020_values.columns = polls2020_values.columns.get_level_values(1)
polls2020_values.columns.name = None

In [11]:
polls2020 = polls2020_descriptive.join(polls2020_values).drop_duplicates()

In [12]:
polls2020['diff'] = polls2020['Biden'] - polls2020['Trump']

In [19]:
population_weight = {'a':0.7, 'v':0.8, 'rv':0.9, 'lv':1.}
polls2020['population_weight'] = polls2020['population'].replace(population_weight)
polls2020 = polls2020.sort_values('population_weight')

In [20]:
polls2020['weight'] = polls2020['sample_size']*polls2020['population_weight']

In [22]:
polls2020 = polls2020.sort_values(['start_date', 'end_date'])
polls2020 = polls2020.reset_index()

In [30]:
polls2020 = polls2020.drop_duplicates(subset=['poll_id', 'pollster_id', 'start_date', 'end_date'], keep='last')

In [39]:
polls2020.to_csv('..\\..\\data\\processed\\polls2020.csv')

In [56]:
polls2020['state'].str.replace(r' CD-[0-9]', '').unique()

array(['Ohio', 'National', 'North Carolina', 'Michigan', 'Iowa',
       'Arizona', 'Texas', 'New Hampshire', 'South Carolina', 'Wisconsin',
       'Pennsylvania', 'Nevada', 'Massachusetts', 'Missouri', 'Florida',
       'North Dakota', 'Kentucky', 'Maine', 'Washington', 'Alaska',
       'Colorado', 'Utah', 'California', 'Virginia', 'Connecticut',
       'Montana', 'Minnesota', 'Georgia', 'New Mexico', 'Alabama',
       'West Virginia', 'Delaware', 'Tennessee', 'Kansas', 'Oklahoma',
       'New Jersey', 'Maryland', 'New York', 'Mississippi', 'Indiana',
       'Nebraska'], dtype=object)

---

---

---

In [79]:
president_approval_polls2020= pd.read_csv('https://projects.fivethirtyeight.com/polls-page/president_approval_polls.csv', 
                                           parse_dates=['start_date', 'end_date', 'created_at'])

president_approval_polls2020.to_csv('..\\..\\data\\raw\\president_approval_polls2020.csv')

In [80]:
print(president_approval_polls2020.columns)

Index(['question_id', 'poll_id', 'state', 'politician_id', 'politician',
       'pollster_id', 'pollster', 'sponsor_ids', 'sponsors', 'display_name',
       'pollster_rating_id', 'pollster_rating_name', 'fte_grade',
       'sample_size', 'population', 'population_full', 'methodology',
       'start_date', 'end_date', 'sponsor_candidate', 'tracking', 'created_at',
       'notes', 'url', 'source', 'yes', 'no'],
      dtype='object')


In [81]:
cols = ['question_id', 'poll_id', 'state', 'politician', 'pollster_id','pollster', 'sample_size', 
        'population', 'start_date', 'end_date', 'created_at', 'url', 'yes', 'no']

president_approval_polls2020 = president_approval_polls2020[cols]
president_approval_polls2020['state'] = president_approval_polls2020['state'].fillna('National')
president_approval_polls2020['politician'] = president_approval_polls2020['politician'].replace({'Donald Trump':'Trump'})
president_approval_polls2020 = president_approval_polls2020.rename({'yes':'approve', 'no':'disapprove'}, axis='columns')

In [83]:
population_weight = {'a':0.7, 'v':0.8, 'rv':0.9, 'lv':1.}
president_approval_polls2020['population_weight'] = president_approval_polls2020['population'].replace(population_weight)
president_approval_polls2020['weight'] = president_approval_polls2020['sample_size']*president_approval_polls2020['population_weight']

president_approval_polls2020 = president_approval_polls2020.drop_duplicates(subset=['poll_id', 'pollster_id', 'start_date', 'end_date'], keep='last')
president_approval_polls2020['diff'] = president_approval_polls2020['approve'] - president_approval_polls2020['disapprove']
president_approval_polls2020 = president_approval_polls2020 = president_approval_polls2020.sort_values(['start_date', 'end_date', 'population_weight'])

president_approval_polls2020.to_csv('..\\..\\data\\processed\\president_approval_polls2020.csv')

,question_id,poll_id,state,politician,pollster_id,pollster,sample_size,population,start_date,end_date,created_at,url,approve,disapprove,population_weight,weight
7525,77265,49253,National,Trump,169,Gallup,1500,a,2017-01-20,2017-01-22,2017-02-06 17:42:00,http://www.gallup.com/poll/201617/gallup-daily...,45.0,45.0,0.7,1050.0
7524,77261,49249,National,Trump,1189,Morning Consult,1992,rv,2017-01-20,2017-01-22,2017-02-06 17:42:00,http://static.politico.com/9b/13/82a3baf542ae9...,46.0,37.0,0.9,1792.8
7522,77599,49426,National,Trump,744,Ipsos,1632,a,2017-01-20,2017-01-24,2017-03-01 17:53:00,http://polling.reuters.com/#poll/CP3_2/,42.1,45.2,0.7,1142.4
7516,77272,49260,National,Trump,396,Quinnipiac University,1190,rv,2017-01-20,2017-01-25,2017-02-06 17:44:00,https://poll.qu.edu/national/release-detail?Re...,36.0,44.0,0.9,1071.0
7523,77274,49262,National,Trump,169,Gallup,1500,a,2017-01-21,2017-01-23,2017-02-06 17:48:00,http://www.gallup.com/poll/201617/gallup-daily...,45.0,46.0,0.7,1050.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,124537,66751,National,Trump,1554,RMG Research,1200,rv,2020-06-04,2020-06-06,2020-06-08 07:43:00,https://scottrasmussen.com/president-trumps-jo...,41.0,57.0,0.9,1080.0
9,124547,66757,National,Trump,568,YouGov,732,rv,2020-06-04,2020-06-06,2020-06-08 14:58:00,https://docs.cdn.yougov.com/a65pdcmcr7/tabs_Tr...,41.0,58.0,0.9,658.8
3,124549,66758,National,Trump,568,YouGov,746,rv,2020-06-05,2020-06-07,2020-06-08 15:46:00,https://docs.cdn.yougov.com/02wog3m3hq/tabs_Tr...,44.0,55.0,0.9,671.4
5,124560,66761,National,Trump,1189,Morning Consult,1991,rv,2020-06-05,2020-06-07,2020-06-08 17:39:00,https://assets.morningconsult.com/wp-uploads/2...,39.0,58.0,0.9,1791.9
